In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=2)
today, yesterday

(datetime.date(2022, 6, 19), datetime.date(2022, 6, 17))

In [2]:
today = today - timedelta(days=2)
wkago = today - timedelta(days=7)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 6, 17), datetime.date(2022, 6, 10))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [3]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

2402

In [4]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AP', 'ASK', 'BCH', 'BCPG', 'BDMS', 'BPP', 'DOHOME', 'GLOBAL', 'IVL', 'JASIF', 'KKP', 'KTC', 'MAKRO', 'NER', 'SCC', 'SIS', 'SSP', 'TISCO', 'TMT', 'TTB'"

In [5]:
len(in_p.split(','))

20

### Get past one year data

In [6]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-06-17' AND name IN ('AP', 'ASK', 'BCH', 'BCPG', 'BDMS', 'BPP', 'DOHOME', 'GLOBAL', 'IVL', 'JASIF', 'KKP', 'KTC', 'MAKRO', 'NER', 'SCC', 'SIS', 'SSP', 'TISCO', 'TMT', 'TTB') 
ORDER BY name, date


In [7]:
df = pd.read_sql(sql, const)
df.shape

(4860, 7)

In [8]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [9]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
AP,B
ASK,B
BCH,S
BCPG,B
BDMS,S
BPP,B
DOHOME,S
GLOBAL,B
IVL,B


### Create monitors from stocks

In [10]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,trade
name,
AP,B
ASK,B
BCH,S
BCPG,B
BDMS,S
BPP,B
DOHOME,S
GLOBAL,B
IVL,B


In [11]:
monitors.shape[0]

20

In [12]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(box_file)

In [13]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,10
1,S,10


In [14]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [15]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(140, 1)

In [16]:
df_merge = pd.merge(orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(140, 3)

In [17]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge


In [18]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(120, 3)

In [19]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
TMT,B,SET
BCH,I,SET100
BDMS,I,SET50
DOHOME,I,SET100
JASIF,I,SET
MAKRO,I,SET
NER,I,SET
SCC,I,SET50
SIS,I,SET


In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [21]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
AP,O
ASK,O
BCPG,O
BPP,O
GLOBAL,O
IVL,O
KKP,O
KTC,O
SSP,O


In [22]:
buy_candidates.shape[0]

10

In [23]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BCH,I
BDMS,I
DOHOME,I
JASIF,I
MAKRO,I
NER,I
SCC,I
SIS,I
TISCO,I


In [24]:
sell_candidates.shape[0]

10